### Referencias: 
- __[blog](http://chriskiehl.com/article/parallelism-in-one-line/)__ 
- __[stackoverflow](https://stackoverflow.com/questions/2846653/how-to-use-threading-in-python)__
    

In [1]:
from multiprocessing.dummy import Pool as ThreadPool

# Recuperar URLs

In [2]:
from urllib.request import urlopen

urls = [
        'http://www.python.org', 
        'http://www.python.org/about/',
        'http://www.onlamp.com/pub/a/python/2003/04/17/metaclasses.html',
        'http://www.python.org/doc/',
        'http://www.python.org/download/',
        'http://www.python.org/getit/',
        'http://www.python.org/community/',
        'https://wiki.python.org/moin/',
        'http://planet.python.org/',
        'https://wiki.python.org/moin/LocalUserGroups',
        'http://www.python.org/psf/',
        'http://docs.python.org/devguide/',
        'http://www.python.org/community/awards/'
        # etc.. 
      ]

# Crea los threads
pool = ThreadPool()

results = pool.map(urlopen, urls)

print("Type of results:", type(results))

print("First 100 bytes:",results[0].read(100))


Type of results: <class 'list'>
First 100 bytes: b'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!-'


In [3]:
import time
results = [] 
t0 = time.time()
for url in urls:
    result = urlopen(url)
    results.append(result)
    
print("Tiempo")
print("Un solo thread:",time.time() - t0,"s")


t0 = time.time()
# # ------- 4 Pool ------- # 
pool = ThreadPool(4) 
results = pool.map(urlopen, urls)

print("4 Threads:",time.time() - t0,"s")


t0 = time.time()
# # ------- 8 Pool ------- # 

pool = ThreadPool(8) 
results = pool.map(urlopen, urls)

print("8 threads:",time.time() - t0,"s")

# # ------- 16 Pool ------- # 

pool = ThreadPool(16) 
results = pool.map(urlopen, urls)
print("16 threads:",time.time() - t0,"s")

Tiempo
Un solo thread: 7.704044342041016 s
4 Threads: 2.0134799480438232 s
8 threads: 1.9450995922088623 s
16 threads: 3.1559293270111084 s


# Imágenes a Escala de Grises. 

## `No ejecutes la celda de abajo desde el notebook`
`Guarda la celda de abajo en un archivo .py.`
<br>
`Ejecuta el archivo desde una terminal o IDE`

In [7]:
%%writefile grayscale.py
from multiprocessing.dummy import Pool as ThreadPool
from PIL import Image
import os
from functools import partial
import time


def get_image_paths(folder):
    return (os.path.join(folder, f) 
    for f in os.listdir(folder) 
      if 'jpeg' in f or "jpg" in f or "png" in f )



def create_grayscale(filename,dir_grayscale): 
    """
    filename: String
        filename of image to convert
    dir_grayscale: String
        dir to save new image in grayscale
    """
    image_grayscale = Image.open(filename)
    image_grayscale = image_grayscale.convert("RGBA")
    image_grayscale= image_grayscale.convert('LA')
    
    base,fname = os.path.split(filename)
    fname, file_extension = os.path.splitext(fname)
    
    fname = fname + ".png"
    
    path_grayscale = os.path.join(dir_grayscale, fname)
    #print("Path to save", path_grayscale)
    image_grayscale.save(path_grayscale)
    
def images_to_grayscale(directory, dir_grayscale):
    """
    directory: String
        directory of images to convert
    """
    images = get_image_paths(directory)
    
    if not os.path.exists(dir_grayscale):
        #If not exist create directory inside working directory
        os.makedirs(dir_grayscale)
    for name_image in images:
        create_grayscale(name_image,dir_grayscale)

    
        
def images_to_grayscale_threading(directory, dir_grayscale):
    """
    directory: String
        directory of images to convert
    """
    images = get_image_paths(directory)
    
    if not os.path.exists(dir_grayscale):
        #If not exist create directory inside working directory
        os.makedirs(dir_grayscale)
    
    create_grayscale_partial = partial(create_grayscale,dir_grayscale = dir_grayscale )
    pool = ThreadPool() #Número de threads igual al número de núcleos en tu procesador.
    pool.map(create_grayscale_partial, images)
    



if __name__ == "__main__":
    directory = "imagenes"
    dir_grayscale ="imagenes_gris"

    t0 = time.time()
    images_to_grayscale(directory, dir_grayscale)
    print("Un solo thread:",time.time() - t0,"s")

    dir_grayscale ="imagenes_gris_threading"
    t0 = time.time()
    images_to_grayscale_threading(directory, dir_grayscale)
    print("Varios threads:",time.time() - t0,"s")



Overwriting grayscale.py
